<a href="https://colab.research.google.com/github/necrodancer/Today-I-learned/blob/master/DL_implementation/DenseNet_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DenseNet PyTorch Implementation
* 2020-02-17에 생성됨.
* DenseNet의 구조를 이해하고, PyTorch에 친숙해지기 위해 HOYA012님의 구현체를 따라해보는 노트북입니다. 아래 참조링크를 통해 이론과 소스코드를 따라가 보았습니다.
* [“DenseNet Tutorial \[1\] Paper Review & Implementation details”](https://hoya012.github.io/blog/DenseNet-Tutorial-1/)
* [“DenseNet Tutorial \[2\] PyTorch Code Implementation”](https://hoya012.github.io/blog/DenseNet-Tutorial-2/)


# Preparation Step

* torch 관련 패키지, 이미지 관련 패키지 import하기.
* 하이퍼파라미터 설정하기.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import glob
import PIL
from PIL import Image
from torch.utils import data  as D
from torch.utils.data.sampler import SubsetRandomSampler
import random
import torchsummary

print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

1.4.0
cuda:0


In [0]:
batch_size = 64
validation_ratio = 0.1
random_seed = 10
initial_lr = 0.1
num_epoch = 1

In [3]:
# transforms.Compose 문서: https://pytorch.org/docs/stable/torchvision/transforms.html

transform_train = transforms.Compose([  # compose 함수는 여러 전처리(변환) 과정을 묶어서 수행한다.
        #transforms.Resize(32),
        transforms.RandomCrop(32, padding=4),  # 랜덤 크롭
        transforms.RandomHorizontalFlip(),  # 가로 뒤집기
        transforms.ToTensor(),  # 텐서화, 0~255를 0~1로 스케일링하는 기능도 있는듯함(https://mjdeeplearning.tistory.com/81)
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]) # (sequence of means, sequence of stds), x,y,z 채널에 대해 각각 mean값과 std 값을 넣어준 것 z = (x-mean)/std
        # 참고로 위 mean, std는 densenet 논문에서 cifar10의 픽셀 평균 및 표준편차를 구한걸 255로 나눈 값이라고 한다.
        # 해당 링크 참조: https://github.com/kuangliu/pytorch-cifar/issues/16

transform_validation = transforms.Compose([
        #transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])
        # validation은 말그대로 평가용이니까 크롭이나 플립을 따로 실시하지 않은 듯하다.

transform_test = transforms.Compose([
        #transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

trainset = torchvision.datasets.CIFAR10(  # datasets.ImageFolder를 사용하면 커스텀 이미지 데이터셋을 사용할 수 있다. https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
    root='./data', train=True, download=True, transform=transform_train)

validset = torchvision.datasets.CIFAR10(  # 똑같은 train set에 대해 변환만 validation_transform을 써줬군
    root='./data', train=True, download=True, transform=transform_validation)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(validation_ratio * num_train))
print(split)

np.random.seed(random_seed)
np.random.shuffle(indices)  # dataset index를 미리 셔플해주는군  

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)  # 주어진 인덱스 대로 데이터를 랜덤 샘플함.  https://pytorch.org/docs/stable/data.html
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(  # 데이터로더에 데이터셋 object, 배치사이즈, 샘플러가 들어가는군.
    trainset, batch_size=batch_size, sampler=train_sampler, num_workers=0
)

valid_loader = torch.utils.data.DataLoader(
    validset, batch_size=batch_size, sampler=valid_sampler, num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=0
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  # 10 classes

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
5000


참고) Cifar10 데이터셋의 구성:

각각의 레이블마다 32×32 크기 이미지인 50,000개의 training 데이터셋, 10,000개의 test 데이터셋이 존재하고, 결과적으로 총 60,000개의 32×32 크기의 이미지로 데이터셋이 구성되어 있다. http://solarisailab.com/archives/2325

# Module Class 생성
* DenseNet 구성 모듈과 대응되는 클래스를 각각 생성
* 각 클래스를 조립하여 전체 구조를 구성


In [0]:
# bn_relu_conv class
class bn_relu_conv(nn.Module):
    def __init__(self, nin, nout, kernel_size, stride, padding, bias=False):
        super(bn_relu_conv, self).__init__()
        self.batch_norm = nn.BatchNorm2d(nin)
        # in-place in python: https://www.tutorialspoint.com/inplace-operator-in-python
        # torch.nn.ReLU: https://pytorch.org/docs/stable/nn.html
        self.relu = nn.ReLU(True)  # True는 in-place연산, 즉 복사없이 바로 값을 연산해서 저장하는 걸(e.g. a += 1) 허용한다는 뜻. 
        self.conv = nn.Conv2d(nin, nout, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)

    def forward(self, x):
        out = self.batch_norm(x)
        out = self.relu(out)
        out = self.conv(out)

        return out

In [0]:
# bottleneck_layer class
"""
DenseNet-BC에서 사용되는 layer
1x1->3x3
drop out 구현
torch.cat을 통해 freature map을 channel-wise로 누적
"""
class bottleneck_layer(nn.Sequential):  # 위와 달리 Sequential을 상속한다.
    def __init__(self, nin, growth_rate, drop_rate=0.2):
        super(bottleneck_layer, self).__init__()

        self.add_module('conv_1x1', bn_relu_conv(nin=nin, nout=growth_rate*4, kernel_size=1, stride=1, padding=0, bias=False))  # 1by1이라서 따로 패딩 안 해줌. growth_rate*4인 이유는 bottle neck 진입시 너무 축약되지 않도록 4를 하이퍼파라미터로써 곱해준 것 같음
        self.add_module('conv_3x3', bn_relu_conv(nin=growth_rate*4, nout=growth_rate, kernel_size=3, stride=1, padding=1, bias=False))  # 3by3 conv인데 출력 크기를 유지하고 싶으니 패딩 1 해준 듯. 채널 수 뻥튀기 방지를 위해 출력은 그냥 growth_rate만큼의 채널로 나옴

        self.drop_rate = drop_rate

    def forward(self, x):
        bottleneck_output = super(bottleneck_layer, self).forward(x)  # sequential을 상속 받아서 forward하면 add_module에 있는 module들이 순차적으로 실행되나보군
        if self.drop_rate > 0:
            bottleneck_output = F.dropout(bottleneck_output, p=self.drop_rate, training=self.training) # nn.functional 안에 dropout이 있군
            # 변수명을 output 대신 out이라 적어서 끊겨 있었다.....
        bottleneck_output = torch.cat((x, bottleneck_output), 1)  # 1번째 채널에 맞춰 concat
        return bottleneck_output

In [0]:
# transition layer class
'''
1x1 conv -> 2x2 avg pool (feature size and channel reduction)
theta로 output feature map의 개수 조절 가능
'''
class Transition_layer(nn.Sequential):
    def __init__(self, nin, theta=0.5):
        super(Transition_layer, self).__init__()

        self.add_module('conv_1x1', bn_relu_conv(nin=nin, nout=int(nin*theta), kernel_size=1, stride=1, padding=0, bias=False))
        self.add_module('avg_pool_2x2', nn.AvgPool2d(kernel_size=2, stride=2, padding=0))

In [0]:
# dense block class
"""
bottleneck layer를 연속적으로 만들어줘서 하나의 dense block을 이루게 한다.
잘 보면 nin_bottleneck_layer 변수를 통해 입력 feature map이 등차 수열을 이루는 걸 확인할 수 있다. (https://hoya012.github.io/blog/DenseNet-Tutorial-1/)
"""
class DenseBlock(nn.Sequential):
    def __init__(self, nin, num_bottleneck_layers, growth_rate, drop_rate=0.2):
        super(DenseBlock, self).__init__()

        for i in range(num_bottleneck_layers):
            nin_bottleneck_layer = nin + growth_rate * i  # 여기 밑에도 nin 입력을 nin으로 해놔서 잘못된 크기가 들어가고 있었다..
            self.add_module('bottleneck_layer_%d'%i,bottleneck_layer(nin=nin_bottleneck_layer, growth_rate=growth_rate, drop_rate=drop_rate))

# DenseNet-BC 구성

In [0]:
class DenseNet(nn.Module):
    def __init__(self, growth_rate=12, num_layers=100, theta=0.5, drop_rate=0.2, num_classes=10):  # 논문에서 제시한 파라미터 값이겠지
        super(DenseNet, self).__init__()
        
        assert (num_layers - 4) % 6 == 0  # 왜 6의 배수가 되어야 할까? 아래 답변 참고
        
        # (num_layers-4)//6 
        num_bottleneck_layers = (num_layers - 4) // 6  # 이해했다. densenet-bc-100-12 자체가 총 layer 개수가 100개고, 이 중 4개를 제외한 나머지가 dense block에 사용된다.
                                                       # 이때 논문에서 각 dense block마다 사용된 bottleneck layer는 16개였다. 그래서 6으로 나눈 거고. 위 assert는 그런 배수를 지키라는 의미
                                                       # https://towardsdatascience.com/densenet-on-cifar10-d5651294a1a8
        # 32 x 32 x 3 --> 32 x 32 x (growth_rate*2)
        self.dense_init = nn.Conv2d(3, growth_rate*2, kernel_size=3, stride=1, padding=1, bias=True)
                
        # 32 x 32 x (growth_rate*2) --> 32 x 32 x [(growth_rate*2) + (growth_rate * num_bottleneck_layers)]
        self.dense_block_1 = DenseBlock(nin=growth_rate*2, num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)

        # 32 x 32 x [(growth_rate*2) + (growth_rate * num_bottleneck_layers)] --> 16 x 16 x [(growth_rate*2) + (growth_rate * num_bottleneck_layers)]*theta
        nin_transition_layer_1 = (growth_rate*2) + (growth_rate * num_bottleneck_layers) 
        self.transition_layer_1 = Transition_layer(nin=nin_transition_layer_1, theta=theta)
        
        # 16 x 16 x nin_transition_layer_1*theta --> 16 x 16 x [nin_transition_layer_1*theta + (growth_rate * num_bottleneck_layers)]
        self.dense_block_2 = DenseBlock(nin=int(nin_transition_layer_1*theta), num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)

        # 16 x 16 x [nin_transition_layer_1*theta + (growth_rate * num_bottleneck_layers)] --> 8 x 8 x [nin_transition_layer_1*theta + (growth_rate * num_bottleneck_layers)]*theta
        nin_transition_layer_2 = int(nin_transition_layer_1*theta) + (growth_rate * num_bottleneck_layers) 
        self.transition_layer_2 = Transition_layer(nin=nin_transition_layer_2, theta=theta)
        
        # 8 x 8 x nin_transition_layer_2*theta --> 8 x 8 x [nin_transition_layer_2*theta + (growth_rate * num_bottleneck_layers)]
        self.dense_block_3 = DenseBlock(nin=int(nin_transition_layer_2*theta), num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)
        
        nin_fc_layer = int(nin_transition_layer_2*theta) + (growth_rate * num_bottleneck_layers) 
        
        # [nin_transition_layer_2*theta + (growth_rate * num_bottleneck_layers)] --> num_classes
        self.fc_layer = nn.Linear(nin_fc_layer, num_classes)
        
    def forward(self, x):
        dense_init_output = self.dense_init(x)  # 위는 그냥 object 또는 method를 축약해놓은 거였네. 실제로 인자를 넣고 하는 건 forward에서 이루어지는군.
        
        dense_block_1_output = self.dense_block_1(dense_init_output)
        transition_layer_1_output = self.transition_layer_1(dense_block_1_output)
        
        dense_block_2_output = self.dense_block_2(transition_layer_1_output)
        transition_layer_2_output = self.transition_layer_2(dense_block_2_output)
        
        dense_block_3_output = self.dense_block_3(transition_layer_2_output)
        
        global_avg_pool_output = F.adaptive_avg_pool2d(dense_block_3_output, (1, 1))   # glabal avg pool도 nn.functional 안에 있구만. 여기서 (1, 1)은 output size 즉 8x8->1x1을 뜻한다.           
        global_avg_pool_output_flat = global_avg_pool_output.view(global_avg_pool_output.size(0), -1)  # (1,1)을 펴주는군

        output = self.fc_layer(global_avg_pool_output_flat)
        
        return output

In [0]:
def DenseNetBC_100_12():
    return DenseNet(growth_rate=12, num_layers=100, theta=0.5, drop_rate=0.2, num_classes=10)

def DenseNetBC_250_24():
    return DenseNet(growth_rate=24, num_layers=250, theta=0.5, drop_rate=0.2, num_classes=10)

def DenseNetBC_190_40():
    return DenseNet(growth_rate=40, num_layers=190, theta=0.5, drop_rate=0.2, num_classes=10)

# 이런 파라미터 관리 좋은 습관인 것 같다.

In [10]:
net = DenseNetBC_100_12()
net.to(device)  # 마지막으로 DenseNet 객체를 생성한 후 이를 .to로 device에 등록해야 한다.

DenseNet(
  (dense_init): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dense_block_1): DenseBlock(
    (bottleneck_layer_0): bottleneck_layer(
      (conv_1x1): bn_relu_conv(
        (batch_norm): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv): Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (conv_3x3): bn_relu_conv(
        (batch_norm): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (bottleneck_layer_1): bottleneck_layer(
      (conv_1x1): bn_relu_conv(
        (batch_norm): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv): Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
   

# Architecture Summary
* torch summary 라이브러리를 통해 자신이 계산한 layer 수와 실제 생성된 모델의 layer 수를 비교할 수 있음
* 모든 연산들의 parameter 수, output shape, total parameter 수, 학습에 필요한 memory size 등을 표시해준다.

In [11]:
torchsummary.summary(net, (3, 32, 32)) # 모델 object와 입력 사이즈를 인자롤 받는다.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 32, 32]             672
       BatchNorm2d-2           [-1, 24, 32, 32]              48
              ReLU-3           [-1, 24, 32, 32]               0
            Conv2d-4           [-1, 48, 32, 32]           1,152
      bn_relu_conv-5           [-1, 48, 32, 32]               0
       BatchNorm2d-6           [-1, 48, 32, 32]              96
              ReLU-7           [-1, 48, 32, 32]               0
            Conv2d-8           [-1, 12, 32, 32]           5,184
      bn_relu_conv-9           [-1, 12, 32, 32]               0
      BatchNorm2d-10           [-1, 36, 32, 32]              72
             ReLU-11           [-1, 36, 32, 32]               0
           Conv2d-12           [-1, 48, 32, 32]           1,728
     bn_relu_conv-13           [-1, 48, 32, 32]               0
      BatchNorm2d-14           [-1, 48,

# Training

In [13]:
criterion = nn.CrossEntropyLoss()  # loss를 criterion이라고 표현하는구나
optimizer = optim.SGD(net.parameters(), lr=initial_lr, momentum=0.9)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[int(num_epoch * 0.5), int(num_epoch * 0.75)], gamma=0.1, last_epoch=-1)
               # last_epoch: The index of last epoch. Default: -1. (https://pytorch.org/docs/stable/optim.html)
for epoch in range(num_epoch):  
    lr_scheduler.step()
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):  # enumerate(sequence, start=0) (https://docs.python.org/ko/3/library/functions.html)
        inputs, labels = data  # 배치 사이즈 만큼 데이터가 나온다. (https://seobway.tistory.com/entry/torchutilsdataDataLoader-%EA%B0%9C%EC%9D%B8-%EC%A0%95%EB%A6%AC)
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Clears the gradients of all optimized torch.Tensor s.

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        show_period = 100
        if i % show_period == show_period-1:    # print every "show_period" mini-batches
            print('[%d, %5d/50000] loss: %.7f' %
                  (epoch + 1, (i + 1)*batch_size, running_loss / show_period))
            running_loss = 0.0
        
        
    # validation part
    correct = 0
    total = 0
    for i, data in enumerate(valid_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        
        _, predicted = torch.max(outputs.data, 1)  # Returns a namedtuple (values, indices)  (https://pytorch.org/docs/stable/torch.html#torch.max)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('[%d epoch] Accuracy of the network on the validation images: %d %%' % 
          (epoch + 1, 100 * correct / total)
         )

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,  6400/50000] loss: 2.2754670
[1, 12800/50000] loss: 2.1744711
[1, 19200/50000] loss: 2.0452563
[1, 25600/50000] loss: 1.9625419
[1, 32000/50000] loss: 1.8791807
[1, 38400/50000] loss: 1.8255380
[1, 44800/50000] loss: 1.7602180
[1 epoch] Accuracy of the network on the validation images: 35 %
Finished Training


# Test
* Test set에 대해 test를 한 뒤 10가지 클래스마다 정확도를 각각 구하고, 또한 전체 정확도를 구하는 과정이 위에 코드로 구현이 되어있습니다.

In [15]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

correct = 0
total = 0

with torch.no_grad():  # for test
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
                
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))            
            
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i])) 

Accuracy of the network on the 10000 test images: 34 %
Accuracy of plane : 32 %
Accuracy of   car : 59 %
Accuracy of  bird : 15 %
Accuracy of   cat : 28 %
Accuracy of  deer : 32 %
Accuracy of   dog : 22 %
Accuracy of  frog : 40 %
Accuracy of horse : 28 %
Accuracy of  ship : 61 %
Accuracy of truck : 28 %


# 후기
1. PyTorch에서 제공하는 DataLoader로 전처리를 가공하는 흐름을 알 수 있었다. 추가적으로 커스텀 데이터를 적용하는 법도 링크를 달아놨다.
2. bnreluconv 모듈부터 bottleneck layer, dense block, 마지막으로 densenet-bc-100-12에 이르는 모델 구현과정을 확인할 수 있었다.
3. 타이핑 중에 변수명이나 파라미터를 잘못 입력하여 네트워크 입출력이 이상하게 나왔다. 그래서 torch summary에서 자꾸 오류가 났고, 출력된 densenet의 입출력과 에러 로그를 대조하면서 대략 2, 3 군데 잘못 입력된 부분을 확인하고 수정했다.
4. 모델 학습 및 평가 코드에 대해서도 살펴볼 수 있었다. 필요한 내용은 그때그때 문서를 참고하고 링크를 달아두었다.
5. torch summary에 모델의 메모리 용량까지 표시해주는 점이 인상 깊었다.
6. 오래 걸릴까봐 일부러 1epoch만 돌렸는데 생각보다 빨리 끝나서 놀랐다. cifar 이미지 크기가 작아서 그렇다고 생각한다. 오랜만에 모델 학습 과정을 지켜보니 감회가 새로웠다.
7. 따라 치다가 복붙 후 주석을 다는 식으로 공부방법을 변경했는데 속도 면에서는 좋았으나 기억에 얼마나 남을지는 모르겠다. 주요 부분은 여전히 타이핑을 해야겠다는 생각이 들었다.